# Data Collecting&Preprocessing

In this project, we have 2 main objectives: prediction of heatwaves and total deaths casued by heatwaves.<br>
For differnt purpores, different datasets are prepared.

In [8]:
import pandas as pd
from datetime import datetime
from meteostat import Monthly,Stations
import pycountry_convert as pc
import warnings

In [14]:
warnings.filterwarnings('ignore')

## Heatwave Data(EMDAT)

In [15]:
urlemdat = 'https://github.com/Themaoyc/MDA/blob/main/Data/emdat%20heatwave.csv?raw=true'
df = pd.read_csv(urlemdat)
df.head()

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1993-0515-AUS,1993,515,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,53.311620,NaN,NaN,NaN,NaN
1,1990-0356-MEX,1990,356,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,48.218797,NaN,NaN,NaN,NaN
2,1990-0085-FRA,1990,85,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,48.218797,NaN,NaN,NaN,NaN
3,1990-0388-KOR,1990,388,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,48.218797,NaN,NaN,NaN,NaN
4,1991-0279-PAK,1991,279,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.260853,NaN,NaN,NaN,NaN


This is a dataset of worldwide heatwaves from 1990 to 2021. 

In [16]:
df.groupby(['Start Month']).count() 

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
Start Month,,,,,,,,,,,,,,,,,,,,,
1,6,6,6,0,6,6,6,6,0,0,...,0,0,0,2,2,6,5,3,2,5
2,2,2,2,0,2,2,2,2,0,0,...,0,0,0,0,0,2,1,0,1,1
3,1,1,1,0,1,1,1,1,0,0,...,0,0,0,0,0,1,1,1,0,1
4,8,8,8,0,8,8,8,8,0,0,...,0,0,0,0,0,8,5,4,1,5
5,16,16,16,0,16,16,16,16,0,0,...,0,0,0,2,2,16,11,10,1,11
6,43,43,43,6,43,43,43,43,0,0,...,0,1,1,4,4,43,35,28,9,35
7,77,77,77,8,77,77,77,77,0,0,...,0,0,0,7,7,77,54,45,10,54
8,23,23,23,0,23,23,23,23,0,0,...,0,1,1,4,4,23,16,13,5,16
11,1,1,1,0,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0


It shows that most of the heatwaves happen in June, July and August.<br>
Some records at November, December and even January are because these countries in the southern hemisphere.<br>
So here, this project will use the temperature from January to May to predict whether a heatwave will happen in the following summer.

In [17]:
#choose the heatwave records started in June, July and August
df_predict=df[df['Start Month'].isin([6,7,8])]
df_predict.head()

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
2,1990-0085-FRA,1990,85,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,48.218797,NaN,NaN,NaN,NaN
3,1990-0388-KOR,1990,388,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,48.218797,NaN,NaN,NaN,NaN
4,1991-0279-PAK,1991,279,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.260853,NaN,NaN,NaN,NaN
5,1990-0373-USA,1990,373,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,265000.0,549578.0,NaN,NaN,48.218797,NaN,NaN,NaN,NaN
6,1993-0432-USA,1993,432,NaN,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,53.311620,NaN,NaN,NaN,NaN


## Collect temperature data

Temperature is collected by the meteostat package.<br>

In [18]:
## the meteostat packages can only work with ISO alpha2 code
#so we need to change the alpha3 code into alpha2
ISO3=df_predict['ISO'].values.tolist()
ISO3=list(set(ISO3))
ISO2=[]
for i in ISO3:
    ISO2.append(pc.country_alpha3_to_country_alpha2(i))

In [19]:
temperaturedata=pd.DataFrame()
for i in ISO2:
    stations = Stations()
    stations = stations.region(i)
    station = stations.fetch(10)
    start = datetime(1990, 1, 1)
    end = datetime(2021, 12, 31)
    T1 = Monthly(station, start, end)
    T1 = T1.fetch()
    T1 = T1.reset_index(inplace=False)
    tavg = T1.groupby('time', as_index=False)['tavg'].mean() 
    tmax = T1.groupby('time', as_index=False)['tmax'].max()
    merge=pd.merge(tavg,tmax)
    merge.insert(1,'ISO',pc.country_name_to_country_alpha3(pc.country_alpha2_to_country_name(i)))
    merge.insert(1,'Country',pc.country_alpha2_to_country_name(i))
    temperaturedata=pd.concat([temperaturedata,merge],axis=0)
temperaturedata.head()

,time,Country,ISO,tavg,tmax
0,1990-01-01,Egypt,EGY,13.475000,NaN
1,1990-02-01,Egypt,EGY,13.850000,NaN
2,1990-03-01,Egypt,EGY,15.766667,NaN
3,1990-04-01,Egypt,EGY,19.350000,NaN
4,1990-05-01,Egypt,EGY,20.900000,NaN


## Process the temperature data

In [20]:
temperaturedata_predict=temperaturedata[['time','Country','ISO','tavg']]
temperaturedata_predict['Month'] = pd.to_datetime(temperaturedata_predict['time']).dt.month_name() 
temperaturedata_predict['time'] = temperaturedata_predict['time'].astype('str')
temperaturedata_predict['Year'] = temperaturedata_predict['time'].str[0:4]
temperaturedata_predict.head()

,time,Country,ISO,tavg,Month,Year
0,1990-01-01,Egypt,EGY,13.475000,January,1990
1,1990-02-01,Egypt,EGY,13.850000,February,1990
2,1990-03-01,Egypt,EGY,15.766667,March,1990
3,1990-04-01,Egypt,EGY,19.350000,April,1990
4,1990-05-01,Egypt,EGY,20.900000,May,1990


In [21]:
temperaturedata_predict=temperaturedata_predict.pivot(index=['Country','ISO','Year'],columns='Month',values='tavg')
temperaturedata_predict=temperaturedata_predict[['January','February','March','April','May']]
temperaturedata_predict.reset_index(inplace=True)
temperaturedata_predict.head()

Month,Country,ISO,Year,January,February,March,April,May
0,Albania,ALB,1990,5.2,9.2,12.8,NaN,19.1
1,Albania,ALB,2000,NaN,NaN,NaN,NaN,NaN
2,Albania,ALB,2001,NaN,10.4,NaN,14.3,NaN
3,Albania,ALB,2002,NaN,NaN,NaN,NaN,NaN
4,Albania,ALB,2003,NaN,NaN,11.2,15.0,22.7


## Drop the missing data

In [22]:
temperaturedata_predict.isna().sum()

Month
Country      0
ISO          0
Year         0
January     34
February    33
March       30
April       28
May         26
dtype: int64

In [23]:
temperaturedata_predict=temperaturedata_predict.dropna()
temperaturedata_predict.head()

Month,Country,ISO,Year,January,February,March,April,May
5,Albania,ALB,2004,6.70,8.50,11.7,15.80,17.60
6,Albania,ALB,2005,6.60,6.40,10.9,14.10,20.40
7,Albania,ALB,2006,5.50,7.80,10.8,15.30,18.80
8,Albania,ALB,2007,7.06,8.32,10.9,14.52,18.76
9,Albania,ALB,2008,5.40,7.02,10.0,13.40,17.94


We then create a Bayern effort variable “Heatwave”. Heatwave=1 means heatwave happened in this country during the year,vice versa.

In [24]:
a=df_predict['Year'].astype('str').str[0:4]
b=df_predict['ISO']
hw = {'Year': a,
        'ISO': b}
hw = pd.DataFrame(hw, columns=['Year', 'ISO'])
hw['Heatwave']=1
hw=hw.dropna()
hw.head()

,Year,ISO,Heatwave
2,1990,FRA,1
3,1990,KOR,1
4,1991,PAK,1
5,1990,USA,1
6,1993,USA,1


In [25]:
temperaturedata_predict=pd.merge(temperaturedata_predict, hw, on=['Year','ISO'],how='left')
temperaturedata_predict=temperaturedata_predict.fillna(value=0)
temperaturedata_predict.head()

,Country,ISO,Year,January,February,March,April,May,Heatwave
0,Albania,ALB,2004,6.70,8.50,11.7,15.80,17.60,1.0
1,Albania,ALB,2005,6.60,6.40,10.9,14.10,20.40,0.0
2,Albania,ALB,2006,5.50,7.80,10.8,15.30,18.80,0.0
3,Albania,ALB,2007,7.06,8.32,10.9,14.52,18.76,1.0
4,Albania,ALB,2008,5.40,7.02,10.0,13.40,17.94,0.0


In [26]:
temperaturedata_predict.isna().sum()

Country     0
ISO         0
Year        0
January     0
February    0
March       0
April       0
May         0
Heatwave    0
dtype: int64

In [135]:
#temperaturedata_predict.to_csv("temperaturedata_predict.csv")

This dataset will be used for predicting heatwaves in later analysis.<br>
Now, we start to prepare the data for predicting the total deaths caused by heatwave.

## GDP data 

In [27]:
urlgdp='https://github.com/Themaoyc/MDA/blob/main/Data/GDP.csv?raw=true'
GDP = pd.read_csv(urlgdp)
GDP.head()

,Country,Scale,Base Year,1950,1951,1952,1953,1954,1955,1956,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,"Afghanistan, Islamic Rep. of",Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1033590.00,1116830.00,1183040.00,1226570.00,1222920.00,1285460.00,1327690.00,1469600.00,1547290.00,NaN
1,Albania,Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1332810.99,1350052.64,1395304.63,1434306.51,1472479.13,1550645.49,1636731.32,1691903.43,1644077.27,NaN
2,Algeria,Millions,NaN,4600.0,5300.0,NaN,NaN,NaN,NaN,NaN,...,16209600.00,16647900.00,17228600.00,16712686.00,17514634.90,18876175.60,20393524.41,20501058.34,18383799.92,NaN
3,"Andorra, Principality of",Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2481.85,2404.54,2462.69,2514.51,2616.86,2655.76,2725.27,2818.81,2507.69,NaN
4,Angola,Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12224949.83,13195003.55,14323859.00,13950290.85,16549600.00,20262300.00,25627742.12,30833488.64,33756462.30,NaN


In [28]:
#select the data from 1990 to 2021
GDP.drop(GDP.columns[2:GDP.columns.get_loc('1990')], axis=1,inplace=True)
GDP.head()

,Country,Scale,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,"Afghanistan, Islamic Rep. of",Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1033590.00,1116830.00,1183040.00,1226570.00,1222920.00,1285460.00,1327690.00,1469600.00,1547290.00,NaN
1,Albania,Millions,NaN,NaN,NaN,NaN,NaN,NaN,334359.13,331324.24,...,1332810.99,1350052.64,1395304.63,1434306.51,1472479.13,1550645.49,1636731.32,1691903.43,1644077.27,NaN
2,Algeria,Millions,554400.0,862132.0,1074695.0,1189724.0,1487400.0,2004990.0,2570000.00,2780200.00,...,16209600.00,16647900.00,17228600.00,16712686.00,17514634.90,18876175.60,20393524.41,20501058.34,18383799.92,NaN
3,"Andorra, Principality of",Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2481.85,2404.54,2462.69,2514.51,2616.86,2655.76,2725.27,2818.81,2507.69,NaN
4,Angola,Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12224949.83,13195003.55,14323859.00,13950290.85,16549600.00,20262300.00,25627742.12,30833488.64,33756462.30,NaN


In [29]:
GDP.groupby(['Scale']).count()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Scale,,,,,,,,,,,,,,,,,,,,,
Millions,198,48,50,55,61,65,95,99,102,108,...,180,186,188,192,195,196,196,195,193,60


### The scale is the same(million dollars) for all the records, so we can drop this colum for simplicity

In [30]:
GDP.drop(['Scale'], axis=1,inplace=True)

In [31]:
GDP=pd.melt(GDP, id_vars=['Country'], value_vars=GDP.columns[1:34], 
            var_name='Year',value_name='GDP(million dollars)', col_level=None)
GDP.head()

,Country,Year,GDP(million dollars)
0,"Afghanistan, Islamic Rep. of",1990,NaN
1,Albania,1990,NaN
2,Algeria,1990,554400.0
3,"Andorra, Principality of",1990,NaN
4,Angola,1990,NaN


### The contry in different tables may be recorded in different ways(i.e. Czech Rep. & Czech Republic), we should rewrite them before merge.

In [32]:
df.loc[df['Country'] == 'United States of America (the)','Country']= 'United States'
df.loc[df['Country'] == 'Netherlands (the)','Country']= 'Netherlands, The'
df.loc[df['Country'] == 'Macedonia (the former Yugoslav Republic of)','Country']= 'North Macedonia, Republic of'
df.loc[df['Country'] == 'Czech Republic (the)','Country']= 'Czech Rep.'
df.loc[df['Country'] == 'Korea (the Republic of)','Country']= 'Korea, Rep. of'
df.loc[df['Country'] == 'Sudan (the)','Country']= 'Sudan'
df.loc[df['Country'] == "Korea (the Democratic People's Republic of)",'Country']= 'Dem. Rep. Korea '
df.loc[df['Country'] == 'Bolivia (Plurinational State of)','Country']= 'Bolivia'
df.loc[df['Country'] == 'Russian Federation (the)','Country']= 'Russian Federation'
df.loc[df['Country'] == 'United Kingdom of Great Britain and Northern Ireland (the)','Country']= 'United Kingdom'
df.loc[df['Country'] == 'Egypt','Country']= 'Egypt, Arab Rep. of'
df.loc[df['Country'] == 'Slovakia','Country']= 'Slovak Rep.'
df.loc[df['Country'] == 'Slovenia','Country']= 'Slovenia, Rep. of'
df.loc[df['Country'] == 'Serbia','Country']= 'Serbia, Rep. of'
df.loc[df['Country'] == 'Serbia','Country']= 'Serbia, Rep. of'

In [33]:
##choose useful columns from emdat data
df=df[["Year","Country","ISO","Associated Dis","Associated Dis2","Appeal","Declaration","Start Year","Start Month","Start Day","End Year",
      "End Month","End Day","Total Deaths","CPI"]]
GDP['Year']=GDP['Year'].astype(int)
merge_gdp = pd.merge(df, GDP, on=['Year','Country'],how='left')
merge_gdp.head()

,Year,Country,ISO,Associated Dis,Associated Dis2,Appeal,Declaration,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars)
0,1993,Australia,AUS,NaN,NaN,NaN,NaN,1993,2,NaN,1993,2,NaN,17.0,53.311620,454964.0
1,1990,Mexico,MEX,Drought,NaN,No,No,1990,4,30.0,1990,6,6.0,380.0,48.218797,NaN
2,1990,France,FRA,NaN,NaN,NaN,NaN,1990,7,24.0,1990,7,24.0,5.0,48.218797,1053546.0
3,1990,"Korea, Rep. of",KOR,NaN,NaN,NaN,NaN,1990,7,29.0,1990,7,29.0,40.0,48.218797,200556300.0
4,1991,Pakistan,PAK,NaN,NaN,No,No,1991,6,1.0,1991,6,15.0,523.0,50.260853,NaN


### Merge with the population data

In [34]:
urlpopulation='https://github.com/Themaoyc/MDA/blob/main/Data/population.csv?raw=true'
population= pd.read_csv(urlpopulation)
population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,107195.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0,694665117.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,39835428.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0,470898870.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0,33933611.0


In [35]:
##similar operations as gdp
population=population.rename(columns={'Country Code':'ISO'})
population.drop(columns=['Country Name','Indicator Name','Indicator Code'], axis=1,inplace=True)
population.drop(population.columns[1:population.columns.get_loc('1990')], axis=1,inplace=True)
population=pd.melt(population, id_vars=['ISO'], value_vars=population.columns[1:34], 
            var_name='Year',value_name='Population', col_level=None)
population['Year']=population['Year'].astype(int)
merge_population = pd.merge(merge_gdp, population, on=['Year','ISO'],how='left')
merge_population.head()

,Year,Country,ISO,Associated Dis,Associated Dis2,Appeal,Declaration,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population
0,1993,Australia,AUS,NaN,NaN,NaN,NaN,1993,2,NaN,1993,2,NaN,17.0,53.311620,454964.0,17667000.0
1,1990,Mexico,MEX,Drought,NaN,No,No,1990,4,30.0,1990,6,6.0,380.0,48.218797,NaN,83943135.0
2,1990,France,FRA,NaN,NaN,NaN,NaN,1990,7,24.0,1990,7,24.0,5.0,48.218797,1053546.0,58235716.0
3,1990,"Korea, Rep. of",KOR,NaN,NaN,NaN,NaN,1990,7,29.0,1990,7,29.0,40.0,48.218797,200556300.0,42869283.0
4,1991,Pakistan,PAK,NaN,NaN,No,No,1991,6,1.0,1991,6,15.0,523.0,50.260853,NaN,110778655.0


## Merge with the health expenditure data

In [36]:
urlhealthexp = 'https://github.com/Themaoyc/MDA/blob/main/Data/health%20expenditure.csv?raw=true'
healthexp = pd.read_csv(urlhealthexp)
healthexp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,44.195249,38.938471,35.958962,51.549980,63.531276,70.451047,...,107.706700,118.269210,113.183637,111.227695,104.541257,99.008506,94.739159,102.410661,96.289480,91.970113
2,Afghanistan,AFG,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,15.803164,17.035744,20.412764,23.890501,...,44.990532,50.853474,51.440262,55.034630,59.008934,58.906528,60.188671,65.706024,69.998604,65.806030
3,Africa Western and Central,AFW,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,20.362792,21.034545,22.119771,34.068933,38.689287,44.220128,...,61.421540,68.678383,71.039947,78.746863,81.804504,73.297775,63.649069,60.968846,59.328810,61.338476
4,Angola,AGO,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,12.998967,28.918121,29.049364,34.875187,49.810741,54.260777,...,96.697014,122.107231,122.185585,143.606873,131.647659,108.582939,95.124977,114.334602,87.310738,71.326004


Only data from 2000-2019 is available.

In [37]:
healthexp=healthexp.rename(columns={'Country Code':'ISO'})
healthexp.drop(columns=['Country Name','Indicator Name','Indicator Code'], axis=1,inplace=True)
healthexp=pd.melt(healthexp, id_vars=['ISO'], value_vars=healthexp.columns[1:21], 
            var_name='Year',value_name='healthexp', col_level=None)
healthexp['Year']=healthexp['Year'].astype(int)
merge_population=merge_population[(merge_population['Year']<=2019)&(merge_population['Year']>=2000)]
merge_healthexp = pd.merge(merge_population, healthexp, on=['Year','ISO'],how='left')
merge_healthexp.head()

,Year,Country,ISO,Associated Dis,Associated Dis2,Appeal,Declaration,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp
0,2000,Bulgaria,BGR,NaN,NaN,NaN,NaN,2000,6,NaN,2000,7,NaN,7.0,63.549547,28124.88,8170172.0,94.497452
1,2000,Cyprus,CYP,NaN,NaN,NaN,NaN,2000,7,NaN,2000,7,NaN,5.0,63.549547,10594.98,943288.0,750.516724
2,2000,Greece,GRC,NaN,NaN,NaN,NaN,2000,7,NaN,2000,7,NaN,27.0,63.549547,141247.28,10805808.0,885.410400
3,2000,Croatia,HRV,NaN,NaN,NaN,NaN,2000,6,NaN,2000,6,NaN,40.0,63.549547,NaN,4468302.0,371.062134
4,2000,Israel,ISR,NaN,NaN,NaN,NaN,2000,7,NaN,2000,7,NaN,NaN,63.549547,540061.20,6289000.0,1513.944702


## Merge with the max temperature

In [38]:
tmax=temperaturedata[['time','Country','ISO','tmax']]
tmax['Month'] = pd.to_datetime(tmax['time']).dt.month_name() 
tmax['time'] = tmax['time'].astype('str')
tmax['Year'] = tmax['time'].str[0:4]
tmax=tmax.pivot(index=['Country','ISO','Year'],columns='Month',values='tmax')
tmax=tmax[['June','July','August']]
tmax.reset_index(inplace=True)
tmax=tmax.dropna()
tmax['tmax']=tmax.max(axis=1, 
                numeric_only=True)
tmax=tmax[['Year','ISO','tmax']]
tmax['Year']=tmax['Year'].astype(int)
deaths_predict=pd.merge(merge_healthexp, tmax, on=['Year','ISO'],how='left')
deaths_predict.head()

,Year,Country,ISO,Associated Dis,Associated Dis2,Appeal,Declaration,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax
0,2000,Bulgaria,BGR,NaN,NaN,NaN,NaN,2000,6,NaN,2000,7,NaN,7.0,63.549547,28124.88,8170172.0,94.497452,34.6
1,2000,Cyprus,CYP,NaN,NaN,NaN,NaN,2000,7,NaN,2000,7,NaN,5.0,63.549547,10594.98,943288.0,750.516724,34.1
2,2000,Greece,GRC,NaN,NaN,NaN,NaN,2000,7,NaN,2000,7,NaN,27.0,63.549547,141247.28,10805808.0,885.410400,33.8
3,2000,Croatia,HRV,NaN,NaN,NaN,NaN,2000,6,NaN,2000,6,NaN,40.0,63.549547,NaN,4468302.0,371.062134,29.9
4,2000,Israel,ISR,NaN,NaN,NaN,NaN,2000,7,NaN,2000,7,NaN,NaN,63.549547,540061.20,6289000.0,1513.944702,41.5


## Dealing with the missing data

In [39]:
deaths_predict.isna().sum()

Year                      0
Country                   0
ISO                       0
Associated Dis          119
Associated Dis2         138
Appeal                  141
Declaration             137
Start Year                0
Start Month               0
Start Day                72
End Year                  0
End Month                 0
End Day                  75
Total Deaths             23
CPI                       0
GDP(million dollars)     11
Population                0
healthexp                 1
tmax                     36
dtype: int64

### Replace the missing Associated Dis, Associated Dis 2, Appeal and Declaration with 0.

In [40]:
deaths_predict=deaths_predict.fillna({"Associated Dis":0,"Associated Dis2":0,"Appeal":0,"Declaration":0})
deaths_predict.head()

,Year,Country,ISO,Associated Dis,Associated Dis2,Appeal,Declaration,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax
0,2000,Bulgaria,BGR,0,0,0,0,2000,6,NaN,2000,7,NaN,7.0,63.549547,28124.88,8170172.0,94.497452,34.6
1,2000,Cyprus,CYP,0,0,0,0,2000,7,NaN,2000,7,NaN,5.0,63.549547,10594.98,943288.0,750.516724,34.1
2,2000,Greece,GRC,0,0,0,0,2000,7,NaN,2000,7,NaN,27.0,63.549547,141247.28,10805808.0,885.410400,33.8
3,2000,Croatia,HRV,0,0,0,0,2000,6,NaN,2000,6,NaN,40.0,63.549547,NaN,4468302.0,371.062134,29.9
4,2000,Israel,ISR,0,0,0,0,2000,7,NaN,2000,7,NaN,NaN,63.549547,540061.20,6289000.0,1513.944702,41.5


In [41]:
#create "Associated Drought" and "Associated Wildfire" 
assdrought = ((deaths_predict["Associated Dis"] == "Drought")|(deaths_predict["Associated Dis2"] == "Drought")).astype("int64")
asswildfire = ((deaths_predict["Associated Dis"] == "Wildfire")|(deaths_predict["Associated Dis2"] == "Wildfire")).astype("int64")
deaths_predict["Associated Drought"] = pd.Series(assdrought, name = 'Associated Drougcreate ht')
deaths_predict["Associated Wildfire"] = pd.Series(asswildfire, name = 'Associated Wildfire')
deaths_predict = deaths_predict.drop(columns=["Associated Dis", "Associated Dis2"])
deaths_predict.head()

,Year,Country,ISO,Appeal,Declaration,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Associated Drought,Associated Wildfire
0,2000,Bulgaria,BGR,0,0,2000,6,NaN,2000,7,NaN,7.0,63.549547,28124.88,8170172.0,94.497452,34.6,0,0
1,2000,Cyprus,CYP,0,0,2000,7,NaN,2000,7,NaN,5.0,63.549547,10594.98,943288.0,750.516724,34.1,0,0
2,2000,Greece,GRC,0,0,2000,7,NaN,2000,7,NaN,27.0,63.549547,141247.28,10805808.0,885.410400,33.8,0,0
3,2000,Croatia,HRV,0,0,2000,6,NaN,2000,6,NaN,40.0,63.549547,NaN,4468302.0,371.062134,29.9,0,0
4,2000,Israel,ISR,0,0,2000,7,NaN,2000,7,NaN,NaN,63.549547,540061.20,6289000.0,1513.944702,41.5,0,0


In [42]:
# "Appeal or Declaration"
appl_dclr = ((deaths_predict["Appeal"] == "Yes")|(deaths_predict["Declaration"] == "Yes")).astype("int64")
deaths_predict["Appeal or Declaration"] = pd.Series(appl_dclr, name="Appeal or Declaration")
deaths_predict = deaths_predict.drop(columns=["Appeal", "Declaration"])
deaths_predict.head()

,Year,Country,ISO,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Associated Drought,Associated Wildfire,Appeal or Declaration
0,2000,Bulgaria,BGR,2000,6,NaN,2000,7,NaN,7.0,63.549547,28124.88,8170172.0,94.497452,34.6,0,0,0
1,2000,Cyprus,CYP,2000,7,NaN,2000,7,NaN,5.0,63.549547,10594.98,943288.0,750.516724,34.1,0,0,0
2,2000,Greece,GRC,2000,7,NaN,2000,7,NaN,27.0,63.549547,141247.28,10805808.0,885.410400,33.8,0,0,0
3,2000,Croatia,HRV,2000,6,NaN,2000,6,NaN,40.0,63.549547,NaN,4468302.0,371.062134,29.9,0,0,0
4,2000,Israel,ISR,2000,7,NaN,2000,7,NaN,NaN,63.549547,540061.20,6289000.0,1513.944702,41.5,0,0,0


### Replace the missing Start Day and End Day with 1.

In [43]:
deaths_predict=deaths_predict.fillna({"Start Day":1,"End Day":1})
deaths_predict.head()

,Year,Country,ISO,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Associated Drought,Associated Wildfire,Appeal or Declaration
0,2000,Bulgaria,BGR,2000,6,1.0,2000,7,1.0,7.0,63.549547,28124.88,8170172.0,94.497452,34.6,0,0,0
1,2000,Cyprus,CYP,2000,7,1.0,2000,7,1.0,5.0,63.549547,10594.98,943288.0,750.516724,34.1,0,0,0
2,2000,Greece,GRC,2000,7,1.0,2000,7,1.0,27.0,63.549547,141247.28,10805808.0,885.410400,33.8,0,0,0
3,2000,Croatia,HRV,2000,6,1.0,2000,6,1.0,40.0,63.549547,NaN,4468302.0,371.062134,29.9,0,0,0
4,2000,Israel,ISR,2000,7,1.0,2000,7,1.0,NaN,63.549547,540061.20,6289000.0,1513.944702,41.5,0,0,0


### The missing max temperature is replaced by the average of other heatwaves.

In [44]:
Tmax=deaths_predict['tmax'].mean()
deaths_predict=deaths_predict.fillna({"tmax":Tmax})
deaths_predict[["Start Day","End Day"]]=deaths_predict[["Start Day","End Day"]].astype(int)
deaths_predict.head()

,Year,Country,ISO,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Associated Drought,Associated Wildfire,Appeal or Declaration
0,2000,Bulgaria,BGR,2000,6,1,2000,7,1,7.0,63.549547,28124.88,8170172.0,94.497452,34.6,0,0,0
1,2000,Cyprus,CYP,2000,7,1,2000,7,1,5.0,63.549547,10594.98,943288.0,750.516724,34.1,0,0,0
2,2000,Greece,GRC,2000,7,1,2000,7,1,27.0,63.549547,141247.28,10805808.0,885.410400,33.8,0,0,0
3,2000,Croatia,HRV,2000,6,1,2000,6,1,40.0,63.549547,NaN,4468302.0,371.062134,29.9,0,0,0
4,2000,Israel,ISR,2000,7,1,2000,7,1,NaN,63.549547,540061.20,6289000.0,1513.944702,41.5,0,0,0


### For the missing value of total deaths, GDP and healthexp , we drop these records.

In [45]:
deaths_predict=deaths_predict.dropna()

we then create a new variable "duration" to record the duration for each heatwave.

In [46]:
start = pd.DataFrame({'Year': deaths_predict['Start Year'],
                   'Month': deaths_predict['Start Month'],
                   'Day': deaths_predict['Start Day']})
start = pd.to_datetime(start)
end = pd.DataFrame({'Year': deaths_predict['End Year'],
                   'Month': deaths_predict['End Month'],
                   'Day': deaths_predict['End Day']})
end = pd.to_datetime(end)
duration = end-start
duration =duration.map(lambda x: x.days)+1
deaths_predict['duration']=duration
deaths_predict.head()

,Year,Country,ISO,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Associated Drought,Associated Wildfire,Appeal or Declaration,duration
0,2000,Bulgaria,BGR,2000,6,1,2000,7,1,7.0,63.549547,28124.88,8170172.0,94.497452,34.6,0,0,0,31
1,2000,Cyprus,CYP,2000,7,1,2000,7,1,5.0,63.549547,10594.98,943288.0,750.516724,34.1,0,0,0,1
2,2000,Greece,GRC,2000,7,1,2000,7,1,27.0,63.549547,141247.28,10805808.0,885.410400,33.8,0,0,0,1
7,2000,Turkey,TUR,2000,6,1,2000,7,1,11.0,63.549547,171494.21,63240196.0,199.499374,27.6,1,0,0,31
9,2000,Romania,ROU,2000,7,1,2000,7,1,6.0,63.549547,80873.10,22442971.0,69.893456,26.5,0,0,0,1


In [47]:
#drop the nusiance columns
deaths_predict=deaths_predict.drop(columns=['Start Year','Start Month','Start Day','End Year','End Month','End Day'])
deaths_predict.head()

,Year,Country,ISO,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Associated Drought,Associated Wildfire,Appeal or Declaration,duration
0,2000,Bulgaria,BGR,7.0,63.549547,28124.88,8170172.0,94.497452,34.6,0,0,0,31
1,2000,Cyprus,CYP,5.0,63.549547,10594.98,943288.0,750.516724,34.1,0,0,0,1
2,2000,Greece,GRC,27.0,63.549547,141247.28,10805808.0,885.410400,33.8,0,0,0,1
7,2000,Turkey,TUR,11.0,63.549547,171494.21,63240196.0,199.499374,27.6,1,0,0,31
9,2000,Romania,ROU,6.0,63.549547,80873.10,22442971.0,69.893456,26.5,0,0,0,1


In [52]:
deaths_predict.isna().sum()

Year                     0
Country                  0
ISO                      0
Total Deaths             0
CPI                      0
GDP(million dollars)     0
Population               0
healthexp                0
tmax                     0
Associated Drought       0
Associated Wildfire      0
Appeal or Declaration    0
duration                 0
dtype: int64

In [278]:
#deaths_predict.to_csv('deaths_predict.csv')